# X1c: TextCNN (No Pretrained) — Phase Structure Discovery (NLP)

**Paper-A (P0009) — X Series: Cross-Domain Generality**

| Item | Value |
|------|-------|
| Dataset | SST-2 (binary sentiment) |
| Model | **TextCNN** (random init, no pretrained weights) |
| Noise | Symmetric, η = 0.4 |
| Trusted ratio | 5% |
| λ grid | {0.10, 0.20, 0.30, 0.40, 0.50, 0.60} |
| Seeds | 0–19 (n=20 per λ) |
| Total runs | **120** |
| Estimated time | ~30–60 min (GPU) |

**Rationale**: DistilBERT's pretrained weights act as a buffer that prevents phase transitions. 
TextCNN with random initialization matches the Vision experiments (ResNet from scratch) 
and should be more susceptible to λ-induced bifurcation.


## 1. Setup

In [ ]:
!pip install -q datasets

import os
import json
import time
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from collections import Counter

warnings.filterwarnings("ignore")
os.environ["TRANSFORMERS_VERBOSITY"] = "error"

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 2. Configuration

In [ ]:
CONFIG = {
    "experiment": "exp_X1c_textcnn",
    "series": "X",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "TextCNN",

    "noise_type": "symmetric",
    "noise_rate": 0.4,
    "trusted_ratio": 0.05,

    "lambda_values": [0.10, 0.20, 0.30, 0.40, 0.50, 0.60],
    "seeds": list(range(20)),

    # Training (longer than BERT fine-tuning since training from scratch)
    "epochs": 20,
    "batch_size": 128,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "max_seq_length": 64,       # shorter for CNN
    "max_vocab_size": 25000,
    "embed_dim": 128,
    "num_filters": 100,
    "filter_sizes": [3, 4, 5],
    "dropout": 0.5,

    "use_bf16": True,
    "max_train_samples": 16000,
    "csv_measure_every_n_steps": 10,

    "output_dir": "X1c_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

n_runs = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
steps_per_epoch = n_used // CONFIG["batch_size"]
total_steps = steps_per_epoch * CONFIG["epochs"]
est_per_run = total_steps * 5 / 1000 + 5  # ~5ms/step for CNN

print(f"Total runs: {n_runs}")
print(f"Steps/epoch: ~{steps_per_epoch}, Total steps/run: ~{total_steps}")
print(f"Est. time/run: ~{est_per_run:.0f}s")
print(f"Est. total: ~{n_runs * est_per_run / 60:.0f} min")


## 3. Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Paper-A_X_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/exp_X1c_textcnn"
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
LOCAL_OUTPUT = CONFIG["output_dir"]
os.makedirs(LOCAL_OUTPUT, exist_ok=True)

def save_to_drive(filename, data):
    local_path = os.path.join(LOCAL_OUTPUT, filename)
    with open(local_path, "w") as f:
        json.dump(data, f, indent=2)
    drive_path = os.path.join(DRIVE_OUTPUT, filename)
    with open(drive_path, "w") as f:
        json.dump(data, f, indent=2)
    return drive_path

def save_figure_to_drive(fig, filename, dpi=300):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        fig.savefig(os.path.join(d, filename), dpi=dpi, bbox_inches="tight", facecolor="white")

print(f"Drive: {DRIVE_OUTPUT}")


## 4. Deterministic Seeding

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## 5. Data Preparation (Manual Tokenization)

In [ ]:
print("Loading SST-2...")
raw_dataset = load_dataset("glue", "sst2")

train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")

# --- Build vocabulary ---
def simple_tokenize(text):
    return text.lower().split()

word_counts = Counter()
for text in train_texts:
    word_counts.update(simple_tokenize(text))

# Special tokens
PAD_IDX = 0
UNK_IDX = 1
vocab = {"<PAD>": PAD_IDX, "<UNK>": UNK_IDX}
for word, count in word_counts.most_common(CONFIG["max_vocab_size"] - 2):
    vocab[word] = len(vocab)

print(f"Vocabulary size: {len(vocab)}")

# --- Encode texts ---
def encode_text(text, max_len):
    tokens = simple_tokenize(text)[:max_len]
    ids = [vocab.get(t, UNK_IDX) for t in tokens]
    # Pad
    ids = ids + [PAD_IDX] * (max_len - len(ids))
    return ids

max_len = CONFIG["max_seq_length"]
train_encoded = torch.tensor([encode_text(t, max_len) for t in train_texts], dtype=torch.long)
test_encoded = torch.tensor([encode_text(t, max_len) for t in test_texts], dtype=torch.long)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

print(f"Train tensor: {train_encoded.shape}, Test tensor: {test_encoded.shape}")


In [ ]:
class SimpleTextDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {"input_ids": self.input_ids[idx], "labels": self.labels[idx]}

test_dataset = SimpleTextDataset(test_encoded, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)
print(f"Test loader: {len(test_dataset)} samples")


## 6. Noise Injection & Trusted Subset

In [ ]:
def prepare_noisy_and_trusted(labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    rng = np.random.RandomState(seed + 10000)
    n = len(labels)
    labels_arr = np.array(labels)
    indices = np.arange(n)

    # Stratified trusted subset
    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels_arr == c]
        n_sel = max(1, int(len(class_idx) * trusted_ratio))
        trusted_indices.extend(rng.choice(class_idx, size=n_sel, replace=False))
    trusted_indices = np.array(sorted(trusted_indices))

    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    # Inject noise
    noisy_labels = labels_arr.copy()
    n_flip = int(len(noisy_indices) * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        candidates = [c for c in range(num_classes) if c != noisy_labels[idx]]
        noisy_labels[idx] = rng.choice(candidates)

    # Subsample
    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = np.sort(rng.choice(noisy_indices, size=max_noisy_samples, replace=False))

    actual_noise = np.mean(noisy_labels[noisy_indices] != labels_arr[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} | Noisy: {len(noisy_indices)} (η_actual={actual_noise:.3f})")
    return torch.tensor(noisy_labels, dtype=torch.long), trusted_indices, noisy_indices

print("Testing...")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.4, 0.05, 2, 0, 16000)
print(f"  OK: {len(ti)} + {len(ni)}")


## 7. TextCNN Model

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_filters,
                 filter_sizes, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.convs = nn.ModuleList([
            nn.Conv1d(embed_dim, num_filters, fs) for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(filter_sizes), num_classes)

    def forward(self, input_ids, labels=None):
        # input_ids: (B, L)
        x = self.embedding(input_ids)          # (B, L, E)
        x = x.transpose(1, 2)                  # (B, E, L)
        pooled = []
        for conv in self.convs:
            h = F.relu(conv(x))                # (B, F, L-fs+1)
            h = F.max_pool1d(h, h.size(2)).squeeze(2)  # (B, F)
            pooled.append(h)
        cat = torch.cat(pooled, dim=1)         # (B, F*len(filter_sizes))
        cat = self.dropout(cat)
        logits = self.fc(cat)                  # (B, C)

        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        return type('Output', (), {'loss': loss, 'logits': logits})()

def create_model(config, vocab_size):
    return TextCNN(
        vocab_size=vocab_size,
        embed_dim=config["embed_dim"],
        num_classes=config["num_classes"],
        num_filters=config["num_filters"],
        filter_sizes=config["filter_sizes"],
        dropout=config["dropout"]
    )

# Test
m = create_model(CONFIG, len(vocab))
print(f"TextCNN params: {sum(p.numel() for p in m.parameters()):,}")
del m


## 8. Dual-Gradient Core

In [ ]:
def get_head_param_names(model):
    return [n for n, _ in model.named_parameters() if "fc" in n]

def extract_grads(model):
    return {n: p.grad.clone() for n, p in model.named_parameters() if p.grad is not None}

def normalize_grad_dict(grads):
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0

def cosine_sim(grads_a, grads_b, param_names=None):
    if param_names:
        keys = [n for n in param_names if n in grads_a and n in grads_b]
    else:
        keys = sorted(set(grads_a) & set(grads_b))
    if not keys:
        return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()

def set_mixed_grad(model, g_struct_n, g_value_n, lam):
    for n, p in model.named_parameters():
        if n in g_struct_n and n in g_value_n:
            p.grad = (1 - lam) * g_struct_n[n] + lam * g_value_n[n]
        elif n in g_struct_n:
            p.grad = (1 - lam) * g_struct_n[n]
        elif n in g_value_n:
            p.grad = lam * g_value_n[n]

@torch.no_grad()
def evaluate(model, test_loader, device):
    model.eval()
    correct = total = 0
    total_loss = 0.0
    n = 0
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        out = model(input_ids=ids, labels=labels)
        preds = out.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += out.loss.item()
        n += 1
    model.train()
    acc = correct / total
    return acc, 1.0 - acc, total_loss / n

print("Core functions defined.")


## 9. Single Run

In [ ]:
def run_single_experiment(lam, seed, config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed, config.get("max_train_samples")
    )

    noisy_ds = SimpleTextDataset(train_encoded[noisy_idx], noisy_labels[noisy_idx])
    trusted_ds = SimpleTextDataset(train_encoded[trusted_idx], noisy_labels[trusted_idx])

    struct_loader = DataLoader(noisy_ds, batch_size=config["batch_size"], shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_ds, batch_size=min(config["batch_size"], len(trusted_ds)),
                              shuffle=True, drop_last=False)

    set_seed(seed)
    model = create_model(config, len(vocab)).to(device)
    model.train()
    head_params = get_head_param_names(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"],
                                 weight_decay=config["weight_decay"])

    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32

    epoch_logs = []
    all_csv = []
    measure_every = config.get("csv_measure_every_n_steps", 10)
    global_step = 0

    for epoch in range(config["epochs"]):
        epoch_loss_s = epoch_loss_v = 0.0
        epoch_csv = []
        n_steps = 0
        value_iter = iter(value_loader)

        for struct_batch in struct_loader:
            optimizer.zero_grad()
            s_ids = struct_batch["input_ids"].to(device)
            s_labels = struct_batch["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_s = model(input_ids=s_ids, labels=s_labels)
            out_s.loss.backward()
            g_struct = extract_grads(model)

            optimizer.zero_grad()
            try:
                vb = next(value_iter)
            except StopIteration:
                value_iter = iter(value_loader)
                vb = next(value_iter)
            v_ids = vb["input_ids"].to(device)
            v_labels = vb["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_v = model(input_ids=v_ids, labels=v_labels)
            out_v.loss.backward()
            g_value = extract_grads(model)

            g_sn, _ = normalize_grad_dict(g_struct)
            g_vn, _ = normalize_grad_dict(g_value)
            set_mixed_grad(model, g_sn, g_vn, lam)
            optimizer.step()

            global_step += 1
            n_steps += 1
            epoch_loss_s += out_s.loss.item()
            epoch_loss_v += out_v.loss.item()

            if global_step % measure_every == 0:
                c = cosine_sim(g_struct, g_value, head_params)
                epoch_csv.append(c)
                all_csv.append(c)

        test_acc, test_error, test_loss = evaluate(model, test_loader, device)
        mean_csv = float(np.mean(epoch_csv)) if epoch_csv else 0.0

        epoch_log = {
            "epoch": epoch + 1, "steps": n_steps,
            "test_acc": round(test_acc, 4), "test_error": round(test_error, 4),
            "test_loss": round(test_loss, 4),
            "loss_struct": round(epoch_loss_s / max(n_steps, 1), 4),
            "loss_value": round(epoch_loss_v / max(n_steps, 1), 4),
            "c_sv_head": round(mean_csv, 4),
        }
        epoch_logs.append(epoch_log)

        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"    Ep {epoch+1}/{config['epochs']}: err={test_error:.4f} acc={test_acc:.4f} c_sv={mean_csv:.4f}")

    final_acc, final_error, _ = evaluate(model, test_loader, device)
    elapsed = time.time() - t_start

    result = {
        "experiment_id": f"X1c-{seed:03d}-lam{lam:.2f}",
        "experiment": config["experiment"],
        "dataset": config["dataset"],
        "model": config["model_name"],
        "noise_type": config["noise_type"],
        "noise_rate": config["noise_rate"],
        "trusted_ratio": config["trusted_ratio"],
        "lambda": lam,
        "seed": seed,
        "test_acc": round(final_acc, 4),
        "test_error": round(final_error, 4),
        "best_acc": round(max(l["test_acc"] for l in epoch_logs), 4),
        "best_error": round(min(l["test_error"] for l in epoch_logs), 4),
        "avg_cos_struct_value": round(float(np.mean(all_csv)) if all_csv else 0.0, 4),
        "total_steps": global_step,
        "epochs": config["epochs"],
        "epoch_logs": epoch_logs,
        "time_seconds": round(elapsed, 1),
    }
    del model, optimizer
    torch.cuda.empty_cache()
    return result

print("Single run function defined (TextCNN).")


## 10. Main Execution Loop

In [ ]:
all_results = []
total_runs = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
run_idx = 0
t_total_start = time.time()

print("=" * 70)
print(f"X1c: TextCNN Coarse λ Sweep — {total_runs} runs")
print(f"  λ: {CONFIG['lambda_values']}, η={CONFIG['noise_rate']}")
print("=" * 70)

for lam in CONFIG["lambda_values"]:
    print(f"\n{'='*55}  λ = {lam:.2f}  {'='*10}")
    lambda_results = []

    for seed in CONFIG["seeds"]:
        run_idx += 1
        print(f"  Run {run_idx}/{total_runs}: λ={lam:.2f} seed={seed}")
        result = run_single_experiment(lam, seed, CONFIG)
        lambda_results.append(result)
        all_results.append(result)

        elapsed = time.time() - t_total_start
        eta = elapsed / run_idx * (total_runs - run_idx)
        print(f"    → err={result['test_error']:.4f} c_sv={result['avg_cos_struct_value']:.4f} "
              f"{result['time_seconds']:.0f}s [ETA: {eta/60:.0f}m]")

    errors = [r["test_error"] for r in lambda_results]
    print(f"  λ={lam:.2f}: mean={np.mean(errors):.4f} ± {np.std(errors):.4f}")
    save_to_drive(f"X1c_lambda{lam:.2f}_results.json", lambda_results)
    print(f"  Saved → Drive ✓")

t_total = time.time() - t_total_start
save_to_drive("X1c_results.json", all_results)
save_to_drive("X1c_config.json", CONFIG)
print(f"\n{'='*70}")
print(f"COMPLETE: {total_runs} runs in {t_total/60:.1f} min")
print(f"Results: {DRIVE_OUTPUT}")
print(f"{'='*70}")


## 11. Analysis & Visualization

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Load
local_path = os.path.join(LOCAL_OUTPUT, "X1c_results.json")
drive_path = os.path.join(DRIVE_OUTPUT, "X1c_results.json")
path = local_path if os.path.exists(local_path) else drive_path
with open(path) as f:
    results = json.load(f)

lambda_data = {}
for r in results:
    lam = r["lambda"]
    if lam not in lambda_data:
        lambda_data[lam] = {"errors": [], "cos": []}
    lambda_data[lam]["errors"].append(r["test_error"])
    lambda_data[lam]["cos"].append(r["avg_cos_struct_value"])

lambdas = sorted(lambda_data.keys())

# --- Summary table ---
print(f"{'λ':>6} | {'mean_err':>10} | {'std_err':>10} | {'min':>8} | {'max':>8} | {'var':>10} | {'c_sv':>8}")
print("-"*70)
for lam in lambdas:
    e = lambda_data[lam]["errors"]
    c = lambda_data[lam]["cos"]
    print(f"{lam:>6.2f} | {np.mean(e):>10.4f} | {np.std(e):>10.4f} | "
          f"{np.min(e):>8.4f} | {np.max(e):>8.4f} | {np.var(e):>10.6f} | {np.mean(c):>8.4f}")

# --- Bimodality check ---
print("\n=== S1 CHECK ===")
for lam in lambdas:
    e = np.array(lambda_data[lam]["errors"])
    stat, p = stats.shapiro(e)
    cv = np.std(e) / np.mean(e)
    rng = np.max(e) - np.min(e)
    flag = " ⚑" if (cv > 0.15 or p < 0.05 or rng > 0.15) else ""
    print(f"  λ={lam:.2f}: CV={cv:.3f} range={rng:.4f} Shapiro_p={p:.4f}{flag}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

ax = axes[0]
for i, lam in enumerate(lambdas):
    jitter = np.random.normal(0, 0.06, len(lambda_data[lam]["errors"]))
    ax.scatter([i+j for j in jitter], lambda_data[lam]["errors"], alpha=0.6, s=25)
bp = ax.boxplot([lambda_data[l]["errors"] for l in lambdas],
                positions=range(len(lambdas)), widths=0.5, patch_artist=True)
for p in bp["boxes"]: p.set_facecolor("#1F77B4"); p.set_alpha(0.3)
ax.set_xticks(range(len(lambdas)))
ax.set_xticklabels([f"{l:.2f}" for l in lambdas])
ax.set_xlabel("λ"); ax.set_ylabel("Test Error")
ax.set_title("(a) Error Distribution"); ax.grid(True, alpha=0.3)

ax = axes[1]
variances = [np.var(lambda_data[l]["errors"]) for l in lambdas]
ax.bar(range(len(lambdas)), [v*10000 for v in variances], color="#FF7F0E", edgecolor="black")
ax.set_xticks(range(len(lambdas)))
ax.set_xticklabels([f"{l:.2f}" for l in lambdas])
ax.set_xlabel("λ"); ax.set_ylabel("Variance (×10⁴)")
ax.set_title("(b) Variance"); ax.grid(True, alpha=0.3)

ax = axes[2]
means_c = [np.mean(lambda_data[l]["cos"]) for l in lambdas]
stds_c = [np.std(lambda_data[l]["cos"]) for l in lambdas]
ax.errorbar(lambdas, means_c, yerr=stds_c, fmt='s-', color="#2CA02C", capsize=5)
ax.axhline(0, color="gray", ls="--", alpha=0.5)
ax.set_xlabel("λ"); ax.set_ylabel("c_sv (head)")
ax.set_title("(c) Gradient Geometry"); ax.grid(True, alpha=0.3)

fig.suptitle("X1c: TextCNN (No Pretrained) — SST-2, η=0.4", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
save_figure_to_drive(fig, "X1c_phase_structure.png")
plt.show()
print("Done.")


## 12. Download

In [ ]:
print(f"Results saved to: {DRIVE_OUTPUT}")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    sz = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f}  ({sz/1024:.1f} KB)")

import shutil
shutil.make_archive("X1c_results", "zip", LOCAL_OUTPUT)
try:
    from google.colab import files
    files.download("X1c_results.zip")
except: pass
